In [ ]:
import os
import pandas as pd
from autoemulate.core.compare import AutoEmulate
from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from ModularCirc import BatchRunner
import torch
figsize = (9, 5)

In [ ]:
param_filename = 'parameters_naghavi_constrained_fixed_T_v_tot'

n_samples = 256

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

parameters_json_file = os.path.join(simulation_out_path, 'parameters.json')

# Load the simulation input parameters
input_params = pd.read_csv(os.path.join(simulation_out_path, f'input_samples_{n_samples}.csv'))

# Load the summary statistics
summary_stats = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

output_to_emulate = 'p_ao_mean'

In [ ]:
# Use ModularCirc's BatchRunner to condense parameter names. This requires setting up the sampler first.
br = BatchRunner()

br.setup_sampler(parameters_json_file)

br._parameters_2_sample

In [ ]:
# Get column names for parameters up to 'lv.k_pas'
parameter_names = list(input_params.columns[:input_params.columns.get_loc('lv.k_pas') + 1])

input_params[parameter_names]

In [ ]:
Y = summary_stats[output_to_emulate].values

In [ ]:
# Turn x into a pytorch tensor
x = torch.tensor(input_params[parameter_names].values, dtype=torch.float32)

# Do the same with Y
y = torch.tensor(Y, dtype=torch.float32)

In [ ]:
print(x.shape, y.shape)

In [ ]:
ae = AutoEmulate(x = x,
                 y = y,
                 models=["GP"])

In [ ]:
best = ae.best_result()
print(best.model_name)

In [ ]:
br._parameters_2_sample

In [ ]:
# Extract parameter names and bounds from the dictionary
parameters_range = br._parameters_2_sample

parameter_names = list(parameters_range.keys())
parameter_bounds = list(parameters_range.values())

In [ ]:
problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds,
}
sa = SensitivityAnalysis(best.model, problem=problem)

In [ ]:
sobol_df = sa.run("sobol")
sobol_df

In [ ]:
sa.plot_sobol(sobol_df, index="ST", figsize=figsize) 